In [90]:
import numpy as np
import random

HIS_tuple.txt format: indexes of dis, sym, herb

In [23]:
with open('../data/HIS_tuple.txt') as dis_dct_file:
    lines = dis_dct_file.readlines()
lines = [line.strip().split() for line in lines]
training = []
for line in lines:
    if len(line) == 3:
        record = [list(map(int, entry.strip()[:-1].split(':'))) for entry in line]
        training.append(record)

In [24]:
# Helper function that parses a text file into a dictionary
def parse_dict(file_name): 
    with open('../data/'+file_name) as file:
        lines = file.readlines()
    lines = [line.strip().split() for line in lines]
    dict = {}
    for line in lines:
        if len(line) == 2:
            dict[int(line[1])] = line[0]
    return dict

In [25]:
sym_dict = parse_dict('sym_dct.txt')
herb_dict = parse_dict('herb_dct.txt')
dis_dict = parse_dict('dis_dct.txt')

In [26]:
num_sym = len(sym_dict)
num_herb = len(herb_dict)
num_dis = len(dis_dict)
num_record = len(training)
print("Number of symptoms: ", num_sym)
print("Number of herbs: ", num_herb)
print("Number of diseases: ", num_dis)
print("Number of records: ",num_record)

Number of symptoms:  15068
Number of herbs:  848
Number of diseases:  1558
Number of records:  9486


In [27]:
count = 0
for record in training:
    count += len(record[1])*len(record[2])
count*num_dis

3040217322

In [28]:
# Initialize symptom popularity dictionary
sym_dict = {}
for i in range(num_sym):
    sym_dict[i+1] = 0

# Initialize herb popularity dictionary
herb_dict = {}
for i in range(num_herb):
    herb_dict[i+1] = 0
    
# Initialize disease popularity dictionary
dis_dict = {}
for i in range(num_dis):
    dis_dict[i+1] = 0

# Loop through training data to fill in the popularity dictionaries
for record in training[:1000]:
    for disease in record[0]:
        dis_dict[disease] += 1
    for symptom in record[1]:
        sym_dict[symptom] += 1
    for herb in record[2]:
        herb_dict[herb] += 1
    

In [119]:
print(dis_dict[2])

44


In [29]:
# Sort each popularity dictionary based on count
sym_list = [(k, sym_dict[k]) for k in sorted(sym_dict, key=sym_dict.get, reverse=True)]
dis_list = [(k, dis_dict[k]) for k in sorted(dis_dict, key=dis_dict.get, reverse=True)]
herb_list = [(k, herb_dict[k]) for k in sorted(herb_dict, key=herb_dict.get, reverse=True)]

# Extract the most popular symptoms, herbs, and diseases based on popularity
key_sym = []
key_herb = []
key_dis = []
# Get top 500 popular symptoms and herbs
for i in range(500):
    key_sym.append(sym_list[i][0])
    key_herb.append(herb_list[i][0])
# Get top 200 popular diseases
for i in range(92):
    key_dis.append(dis_list[i][0])

In [30]:
# Process the training data, eliminate rare diseases, symptoms, and herbs from the training records
key_dis_set = set(key_dis)
key_sym_set = set(key_sym)
key_dis_set = set(key_dis)

key_training = []
for record in training[:1000]:
    new_dis = [x for x in record[0] if x in key_dis_set]
    if len(new_dis) == 0:
        continue
    new_sym = [x for x in record[1] if x in key_sym_set]
    if len(new_sym) == 0:
        continue
    new_herb = [x for x in record[2] if x in key_dis_set]
    if len(new_herb) == 0:
        continue
    key_training.append([new_dis, new_sym, new_herb])
    
len(key_training)

985

In [31]:
tsh_tuples = []
for i in range(len(key_training)):
    record = key_training[i]
    for symptom in record[1]:
        for herb in record[2]:
            tsh_tuples.append((i, symptom, herb))
len(tsh_tuples)

154953

In [35]:
key_num_dis = len(key_dis)
key_num_sym = len(key_sym)
key_num_herb = len(key_herb)
key_num_record = len(key_training)
print("number of diseases: ", key_num_dis)
print("number of symtom: ", key_num_sym)
print("number of herb: ", key_num_herb)
print("number of record: ", key_num_record)

number of diseases:  92
number of symtom:  500
number of herb:  500
number of record:  985
[1, 3, 6, 4, 5, 7, 2, 14, 8, 13, 17, 21, 52, 15, 26, 20, 9, 10, 19, 25, 30, 24, 44, 31, 55, 22, 11, 29, 36, 48, 62, 84, 16, 40, 41, 74, 12, 34, 59, 32, 33, 39, 50, 91, 27, 42, 18, 28, 45, 57, 61, 75, 81, 95, 54, 58, 71, 77, 23, 37, 51, 56, 65, 72, 76, 83, 96, 35, 43, 53, 64, 68, 82, 49, 66, 67, 80, 89, 94, 38, 47, 69, 70, 78, 86, 88, 46, 60, 73, 79, 85, 92]


EM to train the model.
At each step, random pick 10 records as the training data,
then iterator until model converges

In [73]:
def normalizer(input_dict):
    small_mass = 10**-20
    normalizer = 0.0
    for key in input_dict.keys():
        normalizer += input_dict[key]
    normalizer += len(input_dict) * small_mass
    for key in input_dict.keys():
        input_dict[key] = (input_dict[key] + small_mass) / normalizer

In [96]:
# Initialize the model

# P(d)
P_d = {}
for d in key_dis:
    P_d[d] = random.randint(1,3)
normalizer(P_d)

# P(t|d)
P_t_d = {}
for d in key_dis:
    t_dict = {}
    for t in range(len(key_training)):
        t_dict[t] = random.randint(1,3)
    normalizer(t_dict)
    P_t_d[d] = t_dict

# P(s|d)
P_s_d = {}
for d in key_dis:
    s_dict = {}
    for s in key_sym:
        s_dict[s] = random.randint(1,3)
    normalizer(s_dict)
    P_s_d[d] = s_dict

# P(h|d)
P_h_d = {}
for d in key_dis:
    h_dict = {}
    for h in key_herb:
        h_dict[h] = random.randint(1,3)
    normalizer(h_dict)
    P_h_d[d] = h_dict

### EM to train the model

In [104]:
for i in range(20):
    # Compute P(d|s,h,t)
    # Declare a small number added to each entry to prevent 0 probability
    P_d_sht = {}
    for i in range(len(tsh_tuples)):
        (t,s,h) = tsh_tuples[i]
        prob_list = {}
        for d in key_dis:
            prob_list[d] = P_d[d] * P_t_d[d][t] * P_h_d[d][h]
        normalizer(prob_list)
        P_d_sht[(t,s,h)] = prob_list

    old_P_d = np.asarray(list(P_d.values()))

    # Clear out the old distributions from the last iteration
    P_d = dict.fromkeys(P_d, 0.0)
    for d in key_dis:
        P_t_d[d] = dict.fromkeys(P_t_d[d], 0)
        P_s_d[d] = dict.fromkeys(P_s_d[d], 0)
        P_h_d[d] = dict.fromkeys(P_h_d[d], 0)
    # Perform the M step
    for (t,s,h) in P_d_sht.keys():
        d_dict = P_d_sht[(t,s,h)]
        for d in d_dict.keys():
            # update P(d)
            P_d[d] += d_dict[d]
            # update P(t|d)
            P_t_d[d][t] += d_dict[d]
            # update P_s_d
            P_s_d[d][s] += d_dict[d]
            # update P_h_d
            P_h_d[d][h] += d_dict[d]

    # Normalize the new distribution
    normalizer(P_d)
    for d in key_dis:
        normalizer(P_t_d[d])
        normalizer(P_s_d[d])
        normalizer(P_h_d[d])

    new_P_d = np.asarray(list(P_d.values()))
    print(np.abs(old_P_d - new_P_d).max())

0.00020432552659986614
0.00019267849447918498
0.00018069134315051513
0.00016892435572167866
0.0001568236343293325
0.00014449588513092138
0.00013231391091959865
0.0001280708048624138
0.0001266866305197692
0.0001312632242198887
0.00012998606096721252
0.00013018813106019378
0.0001355269870567978
0.00012556965854595634
0.0001243598768862382
0.0001234201700303604
0.00012093552302460883
0.00011603020904740052
0.0001084226424234308
9.903773582598321e-05


In [106]:
# Pickle file for further usage
import pickle

In [110]:
pickle.dump( P_d, open( "../training_result/P_d.p", "wb" ) )
pickle.dump( P_t_d, open( "../training_result/P_t_d.p", "wb" ) )
pickle.dump( P_s_d, open( "../training_result/P_s_d.p", "wb" ) )
pickle.dump( P_h_d, open( "../training_result/P_h_d.p", "wb" ) )

In [112]:
pickle.dump(key_dis, open("../training_result/key_dis", "wb"))
pickle.dump(key_sym, open("../training_result/key_sym", "wb"))
pickle.dump(key_herb, open("../training_result/key_herb", "wb"))

In [113]:
P_s_d

{1: {1: 0.01116737967824906,
  2: 0.023717200971412836,
  3: 0.022114690702643956,
  4: 0.0745440816925315,
  5: 0.04622473558337232,
  6: 0.000305692686263888,
  7: 0.06689223636075305,
  8: 0.0017017192161912164,
  9: 0.0488805417407892,
  10: 0.0017235055533715593,
  11: 9.015673343353513e-06,
  12: 0.01708839040278777,
  13: 0.006791476073851409,
  14: 0.0031629229822772342,
  15: 0.004140112497615877,
  16: 9.786536090090359e-06,
  17: 5.093617028008464e-06,
  18: 0.022174814634247884,
  19: 0.011800664310981762,
  20: 0.01184662323111834,
  21: 1.160508109407037e-07,
  22: 0.021268669864538885,
  23: 0.016903019605002803,
  24: 3.8664840273057747e-07,
  25: 3.877667344945179e-07,
  26: 0.0022429770625815707,
  27: 0.0006791514378179457,
  28: 0.0031580187195767973,
  29: 0.0020662149219438774,
  30: 0.005901539544346955,
  31: 1.0937027403381425e-06,
  32: 0.004023939864832927,
  33: 0.0015688667422441237,
  34: 0.001740360761077599,
  35: 1.8985920676541563e-10,
  36: 0.03234676

In [125]:
print(key_training[120])

[[1, 20], [671, 87, 424, 39, 658, 183, 212, 338, 10, 5, 1270, 196, 2], [4, 69, 13, 46, 2, 7, 88, 47, 33, 11, 1, 9, 15, 18, 17, 61, 24, 12, 58, 91, 32]]


In [120]:
print(key_dis)

[1, 3, 6, 4, 5, 7, 2, 14, 8, 13, 17, 21, 52, 15, 26, 20, 9, 10, 19, 25, 30, 24, 44, 31, 55, 22, 11, 29, 36, 48, 62, 84, 16, 40, 41, 74, 12, 34, 59, 32, 33, 39, 50, 91, 27, 42, 18, 28, 45, 57, 61, 75, 81, 95, 54, 58, 71, 77, 23, 37, 51, 56, 65, 72, 76, 83, 96, 35, 43, 53, 64, 68, 82, 49, 66, 67, 80, 89, 94, 38, 47, 69, 70, 78, 86, 88, 46, 60, 73, 79, 85, 92]
